![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [33]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 

In [34]:
# Data exploration
# Print first, last any sample data points
print(cc_apps.head())
print(cc_apps.tail())
print(cc_apps.sample(10))

print("\n-----Dataframe Info-----")
cc_apps.info()

print("\n-----Columns-----")
print(cc_apps.columns)

  0      1      2  3  4  5  6     7  8  9   10 11   12 13
0  b  30.83  0.000  u  g  w  v  1.25  t  t   1  g    0  +
1  a  58.67  4.460  u  g  q  h  3.04  t  t   6  g  560  +
2  a  24.50  0.500  u  g  q  h  1.50  t  f   0  g  824  +
3  b  27.83  1.540  u  g  w  v  3.75  t  t   5  g    3  +
4  b  20.17  5.625  u  g  w  v  1.71  t  f   0  s    0  +
    0      1       2  3  4   5   6     7  8  9   10 11   12 13
685  b  21.08  10.085  y  p   e   h  1.25  f  f   0  g    0  -
686  a  22.67   0.750  u  g   c   v  2.00  f  t   2  g  394  -
687  a  25.25  13.500  y  p  ff  ff  2.00  f  t   1  g    1  -
688  b  17.92   0.205  u  g  aa   v  0.04  f  f   0  g  750  -
689  b  35.00   3.375  u  g   c   h  8.29  f  f   0  g    0  -
    0      1       2  3  4   5   6       7  8  9   10 11     12 13
175  b  27.00   1.500  y  p   w   v   0.375  t  f   0  g   1065  +
650  b  48.08   3.750  u  g   i  bb   1.000  f  f   0  g      2  -
280  b  21.17   0.875  y  p   c   h   0.250  f  f   0  g    204  -
124  a

In [35]:
print("\n-----Numerical Columns Summary-----")
print(cc_apps.describe())

print("\n-----Categorical Columns Summary-----")
print(cc_apps.describe(include=['object']))



-----Numerical Columns Summary-----
               2           7          10             12
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000

-----Categorical Columns Summary-----
         0    1    3    4    5    6    8    9    11   13
count   690  690  690  690  690  690  690  690  690  690
unique    3  350    4    4   15   10    2    2    3    2
top       b    ?    u    g    c    v    t    f    g    -
freq    468   12  519  519  137  399  361  395  625  383


In [36]:
# Data Preprocessing
# Replace '?' with NaN
cc_apps_nanreplaced = cc_apps.replace('?', np.nan)

# Change column type
cc_apps_nanreplaced[1] = cc_apps_nanreplaced[1].astype(float)

print("\n-----Change 1 column type from object to float-----")
print(cc_apps_nanreplaced.describe())


-----Change 1 column type from object to float-----
               1           2           7          10             12
count  678.000000  690.000000  690.000000  690.00000     690.000000
mean    31.568171    4.758725    2.223406    2.40000    1017.385507
std     11.957862    4.978163    3.346513    4.86294    5210.102598
min     13.750000    0.000000    0.000000    0.00000       0.000000
25%     22.602500    1.000000    0.165000    0.00000       0.000000
50%     28.460000    2.750000    1.000000    0.00000       5.000000
75%     38.230000    7.207500    2.625000    3.00000     395.500000
max     80.250000   28.000000   28.500000   67.00000  100000.000000


In [37]:
# Create a copy of the NaN replacement DataFrame
cc_apps_imputed = cc_apps_nanreplaced.copy()

# Data Preprocessing
# Iterate over each column and impute the most frequent value for object data types and the mean for numeric data types
for col in cc_apps_imputed.columns:
    # Check if the column is of object type
    if cc_apps_imputed[col].dtypes == "object":
        # Impute with the most frequent value
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(
            cc_apps_imputed[col].value_counts().index[0]
        )
    else:
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(cc_apps_imputed[col].mean())

# Dummify the categorical features
cc_apps_encoded = pd.get_dummies(cc_apps_imputed, drop_first=True)

# Extract the last column as your target variable
X = cc_apps_encoded.iloc[:, :-1].values
y = cc_apps_encoded.iloc[:, [-1]].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Instantiate StandardScaler and use it to rescale X_train and X_test
scaler = StandardScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train, y_train)

# Use logreg to predict instances from the training set
y_train_pred = logreg.predict(rescaledX_train)

# Print the confusion matrix of the logreg model
print(confusion_matrix(y_train, y_train_pred))

# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)

# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Fit grid_model to the data
grid_model_result = grid_model.fit(rescaledX_train, y_train)

# Summarize results
best_train_score, best_train_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_train_score, best_train_params))

# Extract the best model and evaluate it on the test set
best_model = grid_model_result.best_estimator_
best_score =  best_model.score(rescaledX_test, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

[[185  19]
 [ 32 226]]
Best: 0.850701 using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  0.8289473684210527
